Se importa aparte << mat73 >>, de lo contrario falla

In [1]:
import mat73

Se importan el resto de las librerías

In [2]:
import numpy as np
import json
import pandas as pd
import random

In [3]:
def finder(dataFrame,pattern):
    for t in range(len(dataFrame)):
        if float(pattern)==float(dataFrame[:][t][0]):
            return(dataFrame[t])

Se llama al archivo con los datos

In [4]:
mat = mat73.loadmat('DATOSTJII_New7.mat')

Inputs

In [5]:
w_length = 24

Columnas en los datos

In [6]:
for key, value in mat.items():
    print(key)

array_shots
data
originalData
signals


Señales, siendo las principales 'BOL5' y 'Densidad2_'

In [7]:
print(mat['signals'])

['ACTON275', 'BOL5', 'Densidad2_', 'ECE7', 'GR', 'GR2', 'HALFAC3', 'IACCEL1', 'RX306']


In [8]:
print('Número de experimentos:',len(mat['array_shots']))

Número de experimentos: 430


Sección para encontrar los tiempos mínimos y máximos para trabajar en los intervalos justos. 

In [9]:
times_per_experiments = {}
descargas_out = []

In [10]:
for descarga in np.arange(len(mat['array_shots'])):
    min_times = []
    max_times = []
    try:
        for signal in np.arange(1,10):
            times = mat['originalData'][descarga][signal][:,0]
            values = mat['originalData'][descarga][signal][:,1]
            min_times.append(np.trunc(100 * np.min(times)) / 100)
            max_times.append(np.trunc(100 * np.max(times)) / 100)
        min_time = np.max(np.array(min_times))
        max_time = np.min(np.array(max_times))
        times_per_experiments[str(mat['originalData'][descarga][0])]=[min_time,max_time]
    except:
        descargas_out.append(str(mat['originalData'][descarga][0]))

Descargas con señales en nulos:

In [11]:
print('N° de descargas con alguna señal completa en nulo:',len(descargas_out))
print('N° de descargas:',len(times_per_experiments))

N° de descargas con alguna señal completa en nulo: 44
N° de descargas: 386


In [12]:
with open("times.json", "w") as outfile:
    json.dump(times_per_experiments, outfile)

In [13]:
keys = list(times_per_experiments.keys())
random.shuffle(keys)

shuffled_times = dict()
for key in keys:
    shuffled_times.update({key: times_per_experiments[key]})

In [14]:
mins = []
for descarga in shuffled_times:
    windows = []
    for c,data in enumerate(finder(mat['originalData'],descarga)):
        if c!=0:
            data[:,0] = np.trunc(100 * data[:,0]) / 100
            df = pd.DataFrame(data, columns = ['time',mat['signals'][c-1]])
            df = df.loc[(df['time']>=float(shuffled_times[str(descarga)][0])) & (df['time']<=float(shuffled_times[str(descarga)][1]))]
            #while True:
            #    if len(df)%float(w_length) == 0: 
            #        break
            #    else: 
            #        df = df.iloc[:len(df)-1]
                    
            windows.append(len(df))
    mins.append(np.argmin(windows))

In [15]:
print(mins)

[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 8, 8, 8, 8, 8, 2, 2, 8, 8, 8, 8, 2, 8, 8, 8, 8, 2, 8, 8, 8, 8, 8, 8, 8, 2, 8, 8, 8, 8, 8, 8, 2, 8, 8, 2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 8, 8, 8, 2, 8, 2, 8, 2, 8, 8, 8, 8, 8, 2, 8, 8, 8, 2, 2, 8, 8, 8, 2, 2, 2, 2, 2, 2, 8, 8, 8, 2, 8, 8, 2, 2, 8, 2, 8, 8, 8, 8, 2, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 8, 8, 2, 2, 8, 2, 2, 2, 8, 2, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 8, 8, 8, 2, 8, 2, 8, 8, 8, 8, 8, 8, 8, 2, 8, 8, 2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 8, 8, 8, 8, 2, 2, 8, 2, 8, 2, 2, 2, 8, 2, 8, 8, 8, 8, 8, 2, 8, 2, 8, 2, 8, 2, 2, 8, 8, 2, 8, 8, 8, 8, 8, 8, 2, 8, 8, 2, 8, 8, 8, 8, 2, 8, 8, 8, 8, 8, 2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 8, 8, 8, 8, 2, 2, 8, 2, 8, 8, 2, 2, 8, 8, 8, 8, 8, 8, 8, 8, 2, 8, 8, 8, 2, 8, 2, 8, 2, 8, 8, 2, 2, 8, 8, 8, 8, 8, 8, 8, 8, 2, 8, 8, 2, 8, 2, 8, 2, 2, 8, 2, 2, 8, 2, 8, 8, 2, 2, 8, 8, 8, 2, 8, 8, 8, 8, 8, 2, 8, 8, 8, 8, 8, 8, 8, 8, 2, 

In [16]:
for d,(descarga,m) in enumerate(zip(shuffled_times,mins)):
    ref_time = finder(mat['originalData'],descarga)[m+1][:,0]
    ref_time = np.trunc(100 * ref_time) / 100
    print(d)
    if d<int(len(shuffled_times)*0.8):
        for c,data in enumerate(finder(mat['originalData'],descarga)):
            if c!=0:
                activate = True
                data[:,0] = np.trunc(100 * data[:,0]) / 100
                df = pd.DataFrame(data, columns = ['time',mat['signals'][c-1]])
                df = df.loc[(df['time']>=float(shuffled_times[str(descarga)][0])) & (df['time']<=float(shuffled_times[str(descarga)][1]))]
                for tm in ref_time:
                    try:
                        df_f = df.loc[df['time']==float(tm)].iloc[:1]
                        if len(df_f)!=0 and activate:
                            activate = False
                            df_2f = df_f
                        elif len(df_f)!=0:
                            df_2f = pd.concat([df_2f, df_f], axis=0)
                    except:
                        continue
                df_2f.columns = ['time', mat['signals'][c-1]]
                if c == 1:
                    df2 = df_2f.sort_index()
                    df2 = df2.reset_index(drop=True)
                else:
                    df1 = df_2f.sort_index()
                    df1 = df1.reset_index(drop=True)
                    df1 = df1[mat['signals'][c-1]]
                    df2 = pd.concat([df2, df1], axis=1)
        if d == 0:
            df_train = df2
        else:
            df_train = pd.concat([df_train,df2],axis=0)
    else:
        for c,data in enumerate(finder(mat['originalData'],descarga)):
            if c!=0:
                activate = True
                data[:,0] = np.trunc(100 * data[:,0]) / 100
                df = pd.DataFrame(data, columns = ['time',mat['signals'][c-1]])
                df = df.loc[(df['time']>=float(shuffled_times[str(descarga)][0])) & (df['time']<=float(shuffled_times[str(descarga)][1]))]
                for tm in ref_time:
                    try:
                        df_f = df.loc[df['time']==float(tm)].iloc[:1]
                        if len(df_f)!=0 and activate:
                            activate = False
                            df_2f = df_f
                        elif len(df_f)!=0:
                            df_2f = pd.concat([df_2f, df_f], axis=0)
                    except:
                        continue
                df_2f.columns = ['time', mat['signals'][c-1]]
                if c == 1:
                    df2 = df_2f.sort_index()
                    df2 = df2.reset_index(drop=True)
                else:
                    df1 = df_2f.sort_index()
                    df1 = df1.reset_index(drop=True)
                    df1 = df1[mat['signals'][c-1]]
                    df2 = pd.concat([df2, df1], axis=1)
        if d == int(len(shuffled_times)*0.8):
            df_test = df2
        else:
            df_test = pd.concat([df_test,df2],axis=0)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [17]:
df_train_no_na = df_train.dropna()
df_test_no_na = df_test.dropna()

In [18]:
print('Train dataset:', len(df_train))
print('Trest dataset:', len(df_test))
print('Train dataset (no na):', len(df_train_no_na))
print('Trest dataset (no na):', len(df_test_no_na))

Train dataset: 7416290
Trest dataset: 1890448
Train dataset (no na): 7301737
Trest dataset (no na): 1861999


In [19]:
df_train_no_na.to_csv('data_train_'+str(w_length)+'.csv',index=False)
df_test_no_na.to_csv('data_test_'+str(w_length)+'.csv',index=False)

In [ ]:
print(df_train.dtypes)
df_train.head()